<a href="https://colab.research.google.com/github/Todah01/Summoners-War-Analysis/blob/main/merge_total_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
# !sudo apt-get install wkhtmltopdf
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
from collections import defaultdict

In [ ]:
def make_multiple_of_five(num):
    if num % 5 == 0:
        return num
    else:
        return num + (5 - num % 5)

# split data func
def count_unique_mon(df_def, check_key):
  dict_save_count = defaultdict(int)

  for idx in range(len(df_def)):
    temp_key = df_def.iloc[idx][check_key]
    dict_save_count[temp_key] += df_def.iloc[idx]["횟수"]

  return dict_save_count

#-------------------------------------------------------------------------------------

# split data func
def organize_df_mon(df_temp, star, df_name):
  starld5_mon_list = []
  starld5_cnt_list = []
  star5_mon_list = []
  star5_cnt_list = []

  # remove duplication
  df_star_mon_total = df_temp.drop('순위', axis = 1)
  df_star_mon_total = pd.DataFrame({"몬스터" : count_unique_mon(df_star_mon_total, "몬스터").keys(), "횟수" : count_unique_mon(df_star_mon_total, "몬스터").values()})
  df_star_mon_total = df_star_mon_total.sort_values("횟수", ascending = False)

  if star == 4:
    return df_star_mon_total

  elif star == 5:
    # divide ld monster from data
    for idx in range(len(df_star_mon_total)):
      temp_str = re.sub('\s+', '', df_star_mon_total.iloc[idx]['몬스터'])
      if temp_str[0] == '빛' or temp_str[0] == '암':
        if temp_str[1:] not in MONSTER_LDNAT4_LIST:
          starld5_mon_list.append(df_star_mon_total.iloc[idx]['몬스터'])
          starld5_cnt_list.append(df_star_mon_total.iloc[idx]['횟수'])
        else:
          star5_mon_list.append(df_star_mon_total.iloc[idx]['몬스터'])
          star5_cnt_list.append(df_star_mon_total.iloc[idx]['횟수'])
      elif temp_str != '0':
        star5_mon_list.append(df_star_mon_total.iloc[idx]['몬스터'])
        star5_cnt_list.append(df_star_mon_total.iloc[idx]['횟수'])

    df_star5_mon_total = pd.DataFrame({'몬스터':star5_mon_list, '횟수':star5_cnt_list})
    df_starld5_mon_total = pd.DataFrame({'몬스터':starld5_mon_list, '횟수':starld5_cnt_list})
    return df_star5_mon_total, df_starld5_mon_total

  else:
    return "Star Error"

#-------------------------------------------------------------------------------------

def make_plt(length, siege_data):
  color_list = []
  for i in range(min(len(siege_data), 10)) :
    if '불' in siege_data['몬스터'].iloc[i] :
      color_list.append('salmon')
    elif '물' in siege_data['몬스터'].iloc[i] :
      color_list.append('dodgerblue')
    elif '풍' in siege_data['몬스터'].iloc[i] :
      color_list.append('khaki')
    elif '빛' in siege_data['몬스터'].iloc[i] :
      color_list.append('whitesmoke')
    elif '암' in siege_data['몬스터'].iloc[i] :
      color_list.append('dimgrey')

  wfw_len = length
  wfw_locator = make_multiple_of_five(int(siege_data['횟수'][0] / 5))

  plt.bar(siege_data['몬스터'].head(wfw_len), siege_data['횟수'].head(wfw_len), color=color_list)
  plt.ylim(0, siege_data['횟수'][0] * 1.2)
  plt.yticks(range(0, siege_data['횟수'][0] + int(siege_data['횟수'][0]/5), wfw_locator))
  plt.xticks(rotation = 30)
  plt.show()

#-------------------------------------------------------------------------------------

# split data func
def count_unique_def(df_def, check_keys):
  dict_save_count = defaultdict(int)

  for idx in range(len(df_def)):
    temp_list = []
    for check_key in check_keys:
      temp_str = re.sub('\s+', '', df_def.iloc[idx][check_key])
      temp_list.append(temp_str)

    temp_key = ' '.join(temp_list)

    if not dict_save_count:
      dict_save_count[temp_key] += int(df_def.iloc[idx]["등장횟수"][:-1]) if str(df_def.iloc[idx]["등장횟수"])[-1] == '회' else int(df_def.iloc[idx]["등장횟수"])
    else:
      for key in list(dict_save_count.keys()):
          tmp_key_list = key.split()
          tmp_sort_dict = sorted(tmp_key_list)
          tmp_sort_list = sorted(temp_list)
          if tmp_sort_dict == tmp_sort_list:
            dict_save_count[key] += int(df_def.iloc[idx]["등장횟수"][:-1]) if str(df_def.iloc[idx]["등장횟수"])[-1] == '회' else int(df_def.iloc[idx]["등장횟수"])
            break
      else:
        dict_save_count[temp_key] += int(df_def.iloc[idx]["등장횟수"][:-1]) if str(df_def.iloc[idx]["등장횟수"])[-1] == '회' else int(df_def.iloc[idx]["등장횟수"])

  return dict_save_count

#-------------------------------------------------------------------------------------

def remove_duplication_in_def(df_temp, star, df_name):
  temp_list_1 = []
  temp = count_unique_def(df_temp, check_keys)
  monster_1_list = [data.split()[0] for data in temp.keys()]
  monster_2_list = [data.split()[1] for data in temp.keys()]
  monster_3_list = [data.split()[2] for data in temp.keys()]
  values = temp.values()
  temp_df = pd.DataFrame({'몬스터1':monster_1_list, '몬스터2':monster_2_list, '몬스터3':monster_3_list, '등장횟수':values})
  temp_df = temp_df.sort_values("등장횟수", ascending = False)
  for idx in range(len(temp_df)):
    temp_list_1.append(temp_df.iloc[idx]['등장횟수'])

  temp_df['등장횟수'] = [str(x) + "회" for x in temp_list_1]

  if star == 4:
    # download 4 star def files
    temp_df.to_csv(df_name +'.csv', index=False, encoding='CP949')
    # files.download(df_name +'.csv')
  elif star == 5:
    # seperate ld def and non ld def
    condition_1 = []
    condition_2 = []
    for idx in range(len(temp_df)):
      for check in check_keys:
        temp_str = re.sub('\s+', '', temp_df.iloc[idx][check])
        if temp_str[0] == '빛' or temp_str[0] == '암':
          if temp_str[1:] not in MONSTER_LDNAT4_LIST:
            condition_1.append(True)
            condition_2.append(False)
            break
      else:
        condition_1.append(False)
        condition_2.append(True)

    df_starld5_def = temp_df.loc[condition_1]
    df_star5_def = temp_df.loc[condition_2]

    # download files
    df_starld5_def.to_csv(df_name + "_ld" + '.csv', index=False, encoding='CP949')
    df_star5_def.to_csv(df_name +'.csv', index=False, encoding='CP949')

    # files.download(df_name + "_ld" + '.csv')
    # files.download(df_name +'.csv')
  else:
    return "Error"

#-------------------------------------------------------------------------------------

def count_unique_def_success(df_def, df_name):
  dict_save_count = defaultdict(lambda:defaultdict(int))
  standard_rate = 0

  # remove duplicate by dictionary
  for idx in range(len(df_def)):
    temp_list = []
    for check_key in check_keys:
      temp_str = re.sub('\s+', '', df_def.iloc[idx][check_key])
      temp_list.append(temp_str)

    temp_key = ' '.join(temp_list)

    if not dict_save_count:
      dict_save_count[temp_key]['총 공격받은 횟수'] += df_def.iloc[idx]['총 공격받은 횟수']
      dict_save_count[temp_key]['방어횟수'] += df_def.iloc[idx]['방어횟수']
    else:
      for key in list(dict_save_count.keys()):
          tmp_key_list = key.split()
          tmp_sort_dict = sorted(tmp_key_list)
          tmp_sort_list = sorted(temp_list)
          if tmp_sort_dict == tmp_sort_list:
            dict_save_count[key]['총 공격받은 횟수'] += df_def.iloc[idx]['총 공격받은 횟수']
            dict_save_count[key]['방어횟수'] += df_def.iloc[idx]['방어횟수']
            break
      else:
        dict_save_count[temp_key]['총 공격받은 횟수'] += df_def.iloc[idx]['총 공격받은 횟수']
        dict_save_count[temp_key]['방어횟수'] += df_def.iloc[idx]['방어횟수']

  temp_total_cnt = 0
  for key in dict_save_count:
    temp_total_cnt += dict_save_count[key]['총 공격받은 횟수']
  standard_rate = (temp_total_cnt / (19 * 2 * 6)) # 서버 당 토너먼트 참가길드 수
  print(standard_rate)

  # set dataframe from dict_save_count
  monster_1_list = []
  monster_2_list = []
  monster_3_list = []
  all_def_cnt = []
  def_success_cnt = []
  def_rate = []
  for data in dict_save_count:
    monster_1_list.append(data.split()[0])
    monster_2_list.append(data.split()[1])
    monster_3_list.append(data.split()[2])
    all_def_cnt.append(str(dict_save_count[data]['총 공격받은 횟수']) + "회")
    def_success_cnt.append(str(dict_save_count[data]['방어횟수']) + "회")
    def_rate.append(str(round((dict_save_count[data]['방어횟수'] / dict_save_count[data]['총 공격받은 횟수']) * 100, 2)) + "%")

  temp_df = pd.DataFrame({'몬스터1':monster_1_list, '몬스터2':monster_2_list, '몬스터3':monster_3_list, '총 공격받은 횟수':all_def_cnt, '방어횟수':def_success_cnt, '방어율':def_rate})
  df_sort_by_def_rate = temp_df[pd.to_numeric(temp_df['총 공격받은 횟수'].str[:-1]) >= standard_rate].sort_values(by=['방어율'], key=lambda x: pd.to_numeric(x.str[:-1]), ascending = False)
  df_sort_by_def_cnt = temp_df.sort_values(by=['방어횟수'], key=lambda x: pd.to_numeric(x.str[:-1]), ascending = False)

  # download files
  df_sort_by_def_rate.to_csv(df_name + "_rate" +'.csv', index=False, encoding='CP949')
  df_sort_by_def_cnt.to_csv(df_name + "_cnt" + '.csv', index=False, encoding='CP949')
  # files.download(df_name + "_rate" +'.csv')
  # files.download(df_name + "_cnt" + '.csv')

  # return temp_df

#Merge Data

In [ ]:
# set the path to the directory containing the CSV files
path = '/content/'

# get a list of all the CSV files in the directory
csv_files = [f for f in os.listdir(path) if f.startswith('star') and f.endswith('.csv')]

# loop through each file and read it into a pandas dataframe
dataframes = []
for filename in csv_files:
    df = pd.read_csv(os.path.join(path, filename), encoding='CP949')
    dataframes.append(df)

df_names = ['df_star4_success_total',  'df_star5_success_total', 'df_star5_mon_total', 'df_star5_def_total','df_starld5_sucess_total', 'df_star4_def_total', 'df_star4_mon_total']
df_server = '_GB'

df_names_plus_server = []

for idx in range(len(df_names)):
  df_names_plus_server.append(df_names[idx]+df_server)

# access each dataframe by index
df_star5_mon_temp = dataframes[0]
df_star4_mon_temp = dataframes[1]
df_star5_def_temp = dataframes[2]
df_star4_success_temp = dataframes[3]
df_star5_success_temp = dataframes[4]
df_star4_def_temp = dataframes[5]
df_starld5_sucess_temp = dataframes[6]

# 5mon/4mon/5def/4success/5success/4def/5ldsuccess

In [ ]:
# Export base file
for idx in range(len(dataframes)):
  dataframes[idx].to_csv(df_names_plus_server[idx] +'.csv', index=False, encoding='CP949')
  files.download(df_names[idx]+ df_server +'.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Merge df
total_dataframes = []
for idx in range(len(df_names_plus_server)):
    df = pd.read_csv(df_names_plus_server[idx] + '.csv', encoding='CP949')
    total_dataframes.append(df)

for idx in range(len(dataframes)):
  df = pd.concat([total_dataframes[idx], dataframes[idx]])
  df.to_csv(df_names[idx] + df_server +'.csv', index=False, encoding='CP949')
  files.download(df_names[idx]+ df_server +'.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
folder_path = '/content/' # 삭제할 파일들이 들어있는 폴더 경로
files = os.listdir(folder_path) # 폴더 내 모든 파일들의 이름 리스트로 저장

for file in files:
    file_path = os.path.join(folder_path, file) # 폴더 경로와 파일 이름 합치기
    if os.path.isfile(file_path): # 파일인 경우에만 삭제
        os.remove(file_path)
        print(f"{file_path}가 성공적으로 삭제되었습니다.")

#Total Data analysis

In [ ]:
MONSTER_LDNAT4_LIST = ['프랑켄', '아수라', '팬텀시프','야만왕', '네오스톤파이터', '요괴무사', '머메이드', '주사위술사', '마법검사', '쿵푸걸', '늑대인간','차크람무녀', '부메랑전사', '호루스', '꿈냥이', '조커']
df_names = ['df_star5_mon_total', 'df_star4_mon_total', 'df_star5_def_total', 'df_star4_def_total', 'df_star5_success_total', 'df_starld5_sucess_total', 'df_star4_success_total']
df_server_list = ['_KR','_AS','_CN','_JP','_GB','_EU']
# df_server_list = ['_KR','_AS','_JP','_GB','_EU'] # except outlier
check_keys = ['몬스터1','몬스터2','몬스터3']
plt.rc('font', family='NanumBarunGothic', size = 15)
plt.rcParams["figure.figsize"] = (14,10)

In [ ]:
total_merge_df_list = []
total_length_list = []

# merge all server df
for df_name in df_names:
  total_length = 0
  total_temp_df_list = []
  for df_server in df_server_list:
    df = pd.read_csv(df_name + df_server + '.csv', encoding = 'CP949')
    total_length += len(df)
    total_temp_df_list.append(df)

  total_merge_df_list.append(pd.concat(total_temp_df_list))
  total_length_list.append(total_length)

# check lost data
for idx in range(len(total_merge_df_list)):
  print(df_names[idx] + " is " + str(len(total_merge_df_list[idx]) == total_length_list[idx]))

df_star5_mon_total is True
df_star4_mon_total is True
df_star5_def_total is True
df_star4_def_total is True
df_star5_success_total is True
df_starld5_sucess_total is True
df_star4_success_total is True


In [ ]:
df_star5, df_starld5 = organize_df_mon(total_merge_df_list[0], 5, df_names[0])
df_star4 = organize_df_mon(total_merge_df_list[1], 4, df_names[1])

In [ ]:
star4_list = ['머메이드', '하늘 무희', '피에레트', '스나이퍼', '요괴무사', '쿵푸걸', '늑대인간', '마법 검사', '하이 엘리멘탈', '광전사', '부메랑 전사', '차크람 무녀',
          '하프술사', '아누비스', '조커', '꿈냥이', '브라우니 요술사', '아수라', 'ROBO', '팬텀시프', '프랑켄', '주사위술사', '야만왕', '잭 오 랜턴', '이누가미', '호루스', '해적선장']

total_count_4 = 0
total_count_5 = 0
for idx in range(len(df_star5)):
  if df_star5['몬스터'].iloc[idx][2:] in star4_list:
    total_count_4 += df_star5['횟수'].iloc[idx]
  else:
    total_count_5 += df_star5['횟수'].iloc[idx]

print(total_count_4)
print(total_count_5)

36009
48110


In [ ]:
make_plt(10, df_starld5)
make_plt(10, df_star5)
make_plt(10, df_star4)

In [ ]:
remove_duplication_in_def(total_merge_df_list[2], 5, df_names[2])
remove_duplication_in_def(total_merge_df_list[3], 4, df_names[3])
count_unique_def_success(total_merge_df_list[4], df_names[4])
count_unique_def_success(total_merge_df_list[5], df_names[5])
count_unique_def_success(total_merge_df_list[6], df_names[6])

33.10526315789474
118.89473684210526
85.42105263157895


#Get average data

In [ ]:
df1 = pd.read_csv('df_star4_success_total_cnt.csv', encoding = 'CP949')
df2 = pd.read_csv('df_star4_success_total_rate.csv', encoding = 'CP949')
df3 = pd.read_csv('df_star5_success_total_cnt.csv', encoding = 'CP949')
df4 = pd.read_csv('df_star5_success_total_rate.csv', encoding = 'CP949')
df5 = pd.read_csv('df_starld5_sucess_total_cnt.csv', encoding = 'CP949')
df6 = pd.read_csv('df_starld5_sucess_total_rate.csv', encoding = 'CP949')
df7 = pd.read_csv('df_star4_def_total.csv', encoding = 'CP949')
df8 = pd.read_csv('df_star5_def_total.csv', encoding = 'CP949')
df9 = pd.read_csv('df_star5_def_total_ld.csv', encoding = 'CP949')

In [ ]:
def get_average_rate(df1):
  sum_root = 0
  sum_roof = 0
  sum_rate = 1
  cnt = 0
  for idx in range(len(df1)):
    if int(df1.iloc[idx]['총 공격받은 횟수'][:-1]) >= 100:
      sum_roof += int(df1.iloc[idx]['방어횟수'][:-1])
      sum_root += int(df1.iloc[idx]['총 공격받은 횟수'][:-1])
      sum_rate += float(df1.iloc[idx]['방어율'][:-1]) * 0.01
      cnt += 1

  print("평균방어율 : ", (sum_roof / sum_root * 100))
  print("평균방어횟수 : ", (sum_rate / cnt))

  return (sum_roof / sum_root * 100), (sum_rate / cnt)

In [ ]:
def remove_duplication(df_mon_list, new_df):
  relation_list = []
  for info in df_mon_list['몬스터']:
    temp_list_3 = []
    temp_list_cnt = 3
    for idx in range(len(new_df)):
      temp_list_str = ""
      for check in check_keys:
        temp_str = re.sub('\s+', '', new_df.iloc[idx][check])
        if temp_str == info:
          temp_list_str += ', '.join([new_df.iloc[idx]['몬스터1'], new_df.iloc[idx]['몬스터2'], new_df.iloc[idx]['몬스터3']
                                    , str(new_df.iloc[idx]['등장횟수'])])
          if len(temp_list_3) != temp_list_cnt:
            temp_list_3.append(temp_list_str)
          else:
            break

          break
    relation_list.append(temp_list_3)
  return relation_list

In [ ]:
def download_mon_list(target_df, mon_df, df_download_name, size):
  cnt_list = []
  temp_list_1 = []
  temp_list_2 = []
  total_ld_cnt = target_df['횟수'].sum()

  for idx in range(len(target_df)):
    temp_list_1.append(target_df.iloc[idx]['횟수'])
    temp_list_2.append(target_df.iloc[idx]['몬스터'])

    cnt_list.append(str(round(int(target_df.iloc[idx]['횟수'])/(total_ld_cnt) * 100, 2))+"%")

  target_df['몬스터'] = [''.join(x.split()) for x in temp_list_2]
  target_df['횟수'] = [str(x) + "회" for x in temp_list_1]
  target_df['점유율'] = cnt_list
  target_df['방덱'] = remove_duplication(target_df ,mon_df)

  target_df.head(size).to_csv(df_download_name, index=False, encoding='CP949')
  # files.download(df_download_name)

In [ ]:
# download_mon_list(df_star4, df7, 'df_star4_mon_list.csv', 10)
download_mon_list(df_star5, df8, 'df_star5_mon_list.csv', 10)
download_mon_list(df_starld5, df9, 'df_star5_ldmon_list.csv', 10)

In [ ]:
total_count = 0
for idx in range(len(df_star4)):
  total_count += int(df_star4['횟수'].iloc[idx][:-1])

print(total_count)

60708


In [ ]:
get_average_rate(df5)

평균방어율 :  13.302013422818792
평균방어횟수 :  0.16151250000000003


(13.302013422818792, 0.16151250000000003)

In [ ]:
total_count = 0
for idx in range(len(df9)):
  total_count += int(df9['등장횟수'].iloc[idx][:-1])

print(total_count)

26059


In [ ]:
#13318 26059
#10364 29254

,몬스터1,몬스터2,몬스터3,등장횟수
0,불피에레트,불이프리트,풍비스트라이더,1505회
1,물하늘무희,불이프리트,풍머메이드,690회
2,불스나이퍼,빛늑대인간,풍광전사,342회
3,물스트라이커,불이프리트,풍비스트라이더,324회
4,불피에레트,물이프리트,풍토템술사,311회
...,...,...,...,...
636,물오라클,불데빌메이든,풍화백,1회
637,풍마도사,물오라클,불팔라딘,1회
638,물이프리트,불그림자술사,풍그림자술사,1회
639,불팔라딘,물이프리트,풍아크엔젤,1회
